In [ ]:
#
# Prototyping for the following ... now it's about optimization
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
from   udist_scatterplots_via_sectors          import UDistScatterPlotsViaSectors
from   udist_scatterplots_via_sectors_tile_opt import UDistScatterPlotsViaSectorsTileOpt
import time
import rtsvg
rt = rtsvg.RACETrack()
import random
# num_of_pts   = [10, 20, 5]
num_of_pts   = [200, 400, 100]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])
for i in range(100):
    x, y = 20*random.random(), 20*random.random()
    _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append('#000000')

_iterations_ = 16

t1 = time.time()
udspvs          = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True)
t2 = time.time()
udspvs_tile_opt = UDistScatterPlotsViaSectorsTileOpt(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True, num_of_tiles=64)
t3 = time.time()
# 351 Points | 512 Iterations | 5.66s Polars Time | 10.68s Polars Tile Opt Time | (M1 Pro 16G)
# 7001 Points | 2 Iterations | 12.30s Polars Time | 4.97s Polars Tile Opt Time (32 tiles) | (M1 Pro 16G)  # After uniquifying the xoyo sectors
# 7001 Points | 2 Iterations | 9.38s Polars Time | 2.95s Polars Tile Opt Time (64 tiles) | (M1 Pro 16G) 
print(f'{len(_xvals_)} Points | {_iterations_} Iterations | {t2-t1:.2f}s Polars Time | {t3-t2:.2f}s Polars Tile Opt Time | (M1 Pro 16G) ')
rt.tile([udspvs.animateIterations(animation_dur="4s"), udspvs_tile_opt.animateIterations(animation_dur="4s")], spacer=10)

In [ ]:
_tiles_ = udspvs_tile_opt.renderStages(rt)
rt.tile([_tiles_[4], _tiles_[7]], spacer=10)